# Исследование надёжности заёмщиков

Заказчик — кредитный отдел банка. Нужно разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. 

Результаты исследования будут учтены при построении модели **кредитного скоринга** — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.

**Ход исследования**
1. Обзор данных.
2. Предобработка данных.
3. Проверка гипотез. Выводы

## Обзор данных

In [1]:
pip install pymystem3

Note: you may need to restart the kernel to use updated packages.


In [2]:
# импортируем все необходимые для работы библиотеки
from collections import Counter

import pandas as pd
from pymystem3 import Mystem

m = Mystem()

In [3]:
# чтение файла с данными и сохранение в переменную
try:
    data = pd.read_csv('/datasets/data.csv')
except:
    data = pd.read_csv('data.csv')

In [4]:
data.head(10)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,-926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,-2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0,-152.779569,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,-6929.865299,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,-2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи


In [5]:
# ознакомились с типом данных в датасете и количеством ненулевых значений
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     19351 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      19351 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


In [6]:
# уникальные значения в колонке 'children'
data['children'].unique()

array([ 1,  0,  3,  2, -1,  4, 20,  5])

In [7]:
# подсчет уникальных значений
data['children'].value_counts()

 0     14149
 1      4818
 2      2055
 3       330
 20       76
-1        47
 4        41
 5         9
Name: children, dtype: int64

In [8]:
# уникальные значения в колонке family_status
data['family_status'].unique()

array(['женат / замужем', 'гражданский брак', 'вдовец / вдова',
       'в разводе', 'Не женат / не замужем'], dtype=object)

**Вывод**  
1. В столбце 'days_employed' много отрицательных вещественных значений. Следовательно, данные в столбце являются некорректными. Есть и положительные вещественные значения, которые не соответствуют адекватному стажу человека за всю его жизнь. Отрицательные значения могли возникнуть если данные заполнялись путем вычитания даты окончания (или настоящего момента) работы из даты (года) начала работы и умножения на 365. 
2. В столбце 'children' есть значения -1 и 20. Количество таких значений равно 47 и 76 соответственно. Строки с такими значениями не единичны, следовательно произошла ошибка при сборе данных. Возможно при обработке данных люди заполняли поле и поставили в графе количество детей дефис 1. А 20, наиболее вероятно, означает количество детей 2.
3. В столбце 'education' используются разные регистры при заполнении данных. Возможны неявные дубликаты.
4. В колонках 'days_employed' и 'total_income' количество ненулевых значений 19351, а строк всего 21525. В колонках 'days_employed' и 'total_income' есть значения NaN.

## Предобработка данных

### Обработка пропусков и некоректных значений

In [9]:
#подсчет пропусков в колонках
data.isna().sum()

children               0
days_employed       2174
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income        2174
purpose                0
dtype: int64

In [10]:
data[data.isnull().any(1)]

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
12,0,NaN,65,среднее,1,гражданский брак,1,M,пенсионер,0,NaN,сыграть свадьбу
26,0,NaN,41,среднее,1,женат / замужем,0,M,госслужащий,0,NaN,образование
29,0,NaN,63,среднее,1,Не женат / не замужем,4,F,пенсионер,0,NaN,строительство жилой недвижимости
41,0,NaN,50,среднее,1,женат / замужем,0,F,госслужащий,0,NaN,сделка с подержанным автомобилем
55,0,NaN,54,среднее,1,гражданский брак,1,F,пенсионер,1,NaN,сыграть свадьбу
...,...,...,...,...,...,...,...,...,...,...,...,...
21489,2,NaN,47,Среднее,1,женат / замужем,0,M,компаньон,0,NaN,сделка с автомобилем
21495,1,NaN,50,среднее,1,гражданский брак,1,F,сотрудник,0,NaN,свадьба
21497,0,NaN,48,ВЫСШЕЕ,0,женат / замужем,0,F,компаньон,0,NaN,строительство недвижимости
21502,1,NaN,42,среднее,1,женат / замужем,0,F,сотрудник,0,NaN,строительство жилой недвижимости


**Note**

Пропуски в колонке 'total_income' важны, так как необходимо узнать зависит ли доход человека от выплаты кредита в срок, поэтому удалить строки с пропущеными значениями мы не можем и их стоит заполнить. Интересно отметить, что пропуски в колонках 'days_employed' и 'total_income' в одних и тех же строках. Пропуски приходятся на разные группы 'income_type'.                               

Пропуски могли возникнуть, напрмер, если человек, по каким-либо причинам, не иммеет официального дохода или стажа работы. Заполнить пропуски нулями нельзя, так как это повлияет на зависимость дохода от выплаты кредита в срок. Среднее значение так же не очень подойдет, так как при заполнении пропущенных значений средним измениться общее среднее для всех группы ('income_type'). Для заполнения используем медианное значение дохода, найденное по типу занятости людей.

Так как стаж работы очень индивидуален для каждого человека, то сложно заполнить пропуски в этой колонке. Поскольку колонка далее не будет использоваться при проверке гипотез, пропущенные значения заменим на 0.

In [11]:
def dob_group(row):
    if 18 <= row.dob_years <= 25:
        return '0-25'
    elif 25<row.dob_years<=35:
        return '25-35'
    elif 35<row.dob_years<=45:
        return '36-45'
    elif 45<row.dob_years<=55:
        return '46-55'
    elif 55<row.dob_years<=65:
        return '56-65'
    elif row.dob_years>65:
        return '65+'
    elif row.dob_years==0:
        return '0'
    return
data['dob_group'] = data.apply(dob_group, axis=1)

In [12]:
# группируем данные и находим медианное значение дохода для каждой группы
median_total_income = data.groupby(['income_type', 'education', 'dob_group'], as_index=False)['total_income'].median()
median_total_income

,income_type,education,dob_group,total_income
0,безработный,Высшее,36-45,202722.511368
1,безработный,среднее,25-35,59956.991984
2,в декрете,СРЕДНЕЕ,36-45,53829.130729
3,госслужащий,ВЫСШЕЕ,0-25,262212.588221
4,госслужащий,ВЫСШЕЕ,25-35,193544.775495
...,...,...,...,...
235,сотрудник,среднее,56-65,141685.195480
236,сотрудник,среднее,65+,153651.057841
237,сотрудник,ученая степень,36-45,115949.039788
238,сотрудник,ученая степень,56-65,268411.214536


In [13]:
data.education = data.education.str.lower()

median_income = data.pivot_table(index=['income_type', 'gender'],
                               columns='education', values='total_income', aggfunc='median')

In [14]:
median_income.head()

education                  высшее      начальное  неоконченное высшее  \
income_type gender                                                      
безработный F       202722.511368            NaN                  NaN   
            M                 NaN            NaN                  NaN   
в декрете   F                 NaN            NaN                  NaN   
госслужащий F       157841.438279   89618.962330        136599.129610   
            M       214706.640963  190966.659534        177320.628254   

education                 среднее  ученая степень  
income_type gender                                 
безработный F                 NaN             NaN  
            M        59956.991984             NaN  
в декрете   F        53829.130729             NaN  
госслужащий F       125831.806597   111392.231107  
            M       169651.956756             NaN

In [15]:
# получим значение медианы
median_income.loc[('студент', 'M')]['высшее']

98201.62531401133

Напишем функцию, которая позволит извлекать нужное значение медианы

In [16]:
def super_fillna_func(income_type, gender, education):
    '''
    Находит в таблице median_income нужную медиану.
    '''
    try:
        return median_income.loc[(income_type, gender)][education]
    except:
        return 0
    
print(super_fillna_func('студент', 'M','высшее') ) 

98201.62531401133


Заполним пропуски:

In [21]:
# Запишем в новый столбец.
data['new_income'] = data.apply(lambda row: super_fillna_func(row['income_type'],
                                                                                row['gender'], row['education']),
                                                  axis=1)

# Пандас сам заменит пропуски значениями из нового столбца в той же строке.
data['total_income'] = data['total_income'].fillna(data['new_income'])

In [22]:
# замена отрицательного значения в столбце 'children' и замена значения 20 на 2
data.loc[data.children < 0, 'children'] = data.children * -1
data['children'] = data['children'].replace(20, 2)
data['children'].unique()

array([1, 0, 3, 2, 4, 5])

In [23]:
# замена отрицательных значений в столбце 'days_employed'
data['days_employed'] = data['days_employed'].abs()
data['days_employed']

0          8437.673028
1          4024.803754
2          5623.422610
3          4124.747207
4        340266.072047
             ...      
21520      4529.316663
21521    343937.404131
21522      2113.346888
21523      3112.481705
21524      1984.507589
Name: days_employed, Length: 21525, dtype: float64

In [24]:
# Если возрост выхода на пенсию 65, то приблизительно возможный стаж работы (65-18)*365 = 17155.
data.loc[data.days_employed > 20000, 'days_employed'] = data.days_employed / 100
data['days_employed']

0        8437.673028
1        4024.803754
2        5623.422610
3        4124.747207
4        3402.660720
            ...     
21520    4529.316663
21521    3439.374041
21522    2113.346888
21523    3112.481705
21524    1984.507589
Name: days_employed, Length: 21525, dtype: float64

**Вывод**
1. Пропуски в столбце total_income заполнены медианным значением дохода исходя из типа деятельности человека, образования, возрастной группы;
2. Исправлены некорректные значения в столбцах days_employed и children;
3. Заменены некорректные значния в столбце children. (почему такие значения могли возникнуть, я попыталась объяснить в выводе, в теме "Обзор данных").



### Замена типа данных

In [25]:
# замена типа данных в колонке 
try:
    data[['days_employed',
          'education_id',
          'children', 
          'family_status_id',
          'debt']] = data[['days_employed',
                           'education_id',
                           'children', 
                           'family_status_id',
                           'debt']].astype('int')
except:
    print('Возникла ошибка с приведением типов')
data.info()

Возникла ошибка с приведением типов
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 14 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     19351 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      21525 non-null  float64
 11  purpose           21525 non-null  object 
 12  dob_group         21525 non-null  object 
 13  new_income        21525 non-null  float64
dtypes: float64(3), int64(5), object(6)
memory usage: 2.3+ MB


**Вывод**

Для замены типа данных использовали метод `astype()`. Метод `to_numeric()` в данном случает не подходит, так как он больше применим для преобразования типа `object` в один из типов `float` или `int`. 

### Обработка дубликатов

In [26]:
#обработка неявных дубликатов. Смотрим уникальные значения столбца
data['education'].unique()

array(['высшее', 'среднее', 'неоконченное высшее', 'начальное',
       'ученая степень'], dtype=object)

In [27]:
# приводим все значения в столбце к нижнему регистру
data['education'] = data['education'].str.lower()
data['education'].unique()

array(['высшее', 'среднее', 'неоконченное высшее', 'начальное',
       'ученая степень'], dtype=object)

In [28]:
#считаем количество дубликотов
data.duplicated().sum()

71

Удалять не нужно, так как нет уникальных идентификаторов пользователей

**Вывод**

1. Проверили наличие дубликатов, но из-за отсутствия уникального идентификатора пользователя, невозможно сказать полные это дубликаты или нет.

Дубликаты могли возникнуть в следующей ситуации: при внесении данных была потеряна связь с сервером, данные сохранились в базу данных, но программа не получила ответ подтверждающий сохранение данных и эти же данные были внесены еще раз.

### Лемматизация

In [29]:
#лематизация целей кредита и подсчет лемм
purpose_group = data['purpose'].tolist()
purpose_group_str = ' '.join(purpose_group)
purpose_group_lem = m.lemmatize(purpose_group_str)
purpose_group_lem_count = Counter(purpose_group_lem)
purpose_group_lem_count

Counter({'покупка': 5912,
         ' ': 55201,
         'жилье': 4473,
         'приобретение': 462,
         'автомобиль': 4315,
         'дополнительный': 909,
         'образование': 4022,
         'сыграть': 774,
         'свадьба': 2348,
         'операция': 2610,
         'с': 2924,
         'на': 2233,
         'проведение': 777,
         'для': 1294,
         'семья': 641,
         'недвижимость': 6367,
         'коммерческий': 1315,
         'жилой': 1233,
         'строительство': 1881,
         'собственный': 635,
         'подержать': 858,
         'свой': 2235,
         'со': 630,
         'заниматься': 908,
         'сделка': 944,
         'получение': 1316,
         'высокий': 1375,
         'подержанный': 110,
         'профильный': 436,
         'сдача': 653,
         'ремонт': 612,
         '\n': 1})

**Вывод**

С помощью лемматизации определили ключевые слова для последующей категоризации данных. Выделили следующие категории: недвижимость, свадьба, автомобиль, образование, ремонт.

### Категоризация данных

In [30]:
#функция одной строки, категоризация целей кредита
def goal_credit(row):
    lem = m.lemmatize(row.purpose)
    if ('ремонт' in lem) and ('жилье' in lem):
        return 'ремонт'
    elif ('жилье' in lem) or ('жилой'in lem) or ('недвижимость'in lem) or ('строительство' in lem):
        return 'недвижимость'
    elif 'свадьба' in lem:
        return 'свадьба'
    elif ('автомобиль' in lem):
        return 'автомобиль'
    elif ('образование' in lem):
        return 'образование'
    
    return
data['goal_credit'] = data.apply(goal_credit, axis=1)
data.head(10)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,dob_group,new_income,goal_credit
0,1,8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья,36-45,153691.851098,недвижимость
1,1,4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля,36-45,124127.090016,автомобиль
2,0,5623.422610,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья,25-35,156286.402929,недвижимость
3,3,4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование,25-35,156286.402929,образование
4,0,3402.660720,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу,46-55,112790.975121,свадьба
5,0,926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья,25-35,233064.883613,недвижимость
6,0,2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем,36-45,189893.464621,недвижимость
7,0,152.779569,50,среднее,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование,46-55,156286.402929,образование
8,2,6929.865299,35,высшее,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы,25-35,153691.851098,свадьба
9,0,2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи,36-45,156286.402929,недвижимость


In [31]:
# категоризация людей по количеству детей
def children_group(row):
    if row.children == 0:
        return 'бездетный'
    elif (row.children == 1) or (row.children == 2):
        return 'малодетный'
    elif row.children > 2:
        return 'многодетный'
    return
data['children_group'] = data.apply(children_group, axis=1)
#data['children_group'].isna().sum() проверили нет ли NaN

**Note**

Исходя из информации, предоставленной интернет ресурсами: В Москве для причисления к среднему классу необходимо получать от 121 тыс. рублей. Для регионов минимальная зарплата для причисления к среднему классу составляет 60 тыс. рублей.

In [32]:
# категоризация по уровню дохода
data['total_income'].value_counts(bins=6)
def total_income_group(row):
    if row.total_income <= 40000:
        return 'низкий уровень дохода'
    elif 40000 < row.total_income <= 60000:
        return 'средний доход кат.1'
    elif 60000< row.total_income <= 100000:
        return 'средний доход кат.2'
    elif 100000< row.total_income <= 140000:
        return 'средний доход кат.3'
    elif 140000< row.total_income <= 200000:
        return 'средний доход кат.4'
    else:
        return 'высокий уровень дохода'
    return
data['total_income_group'] = data.apply(total_income_group, axis=1)
#data['total_income_group'].isna().sum() проверили нет ли NaN

In [33]:
data['total_income_group'].value_counts()

средний доход кат.4       6314
средний доход кат.3       5601
высокий уровень дохода    5144
средний доход кат.2       3659
средний доход кат.1        681
низкий уровень дохода      126
Name: total_income_group, dtype: int64

**Вывод**

Используя метод `apply()` и функцию одной строки выполнили следующее:
1. Добавили столбец `goal_credit`. В данном столбце, в зависимости от цели кредита в сторке, назначена одна из пяти категорий: недвижимость, свадьба, ремонт, автомобиль, образование.
2. Выделено три категории людей в зависимости от количества детей: малодетные, бездетные, многодетные. Создан новый столбец `children_group` отражающий принадлежность людей к той или иной группе.
3. Выделено три категории людей в зависимости от их дохода в месяц: низкий уровень дохода, средний уровень дохода, высокий уровень дохода. Создан новый столбец `total_income_group` отрожающий принадлежность людей к той или иной группе.

## Шаг 3. Итоги

- Есть ли зависимость между наличием детей и возвратом кредита в срок?

In [37]:
def my_mean(x): return '{:.2%} '.format(x.mean())

data.groupby('children_group')['debt'].agg(['count', 'sum', my_mean])

,count,sum,my_mean
children_group,,,
бездетный,14149,1063,7.51%
малодетный,6996,647,9.25%
многодетный,380,31,8.16%


**Вывод**

Около 7,5% людей не имеющих детей имеют долг по кредиту. 9,3% малодетных и 8,2% многодетных людей имеют долг по кредиту. Люди имеющие 1-2 ребенка чаще имеют задолженость по кредиту.


- Есть ли зависимость между семейным положением и возвратом кредита в срок?

In [38]:
data.groupby('family_status')['debt'].agg(['count', 'sum', my_mean])

,count,sum,my_mean
family_status,,,
Не женат / не замужем,2813,274,9.74%
в разводе,1195,85,7.11%
вдовец / вдова,960,63,6.56%
гражданский брак,4177,388,9.29%
женат / замужем,12380,931,7.52%


**Вывод** 

Люди из групп 'не женат / не замужем' и 'гражданский брак' имеют больше задолженостей по кредитам.

In [39]:
data_pivot_children_family = data.pivot_table(index=['children_group','family_status'], values='debt',
                                              aggfunc= my_mean)
data_pivot_children_family

debt
children_group family_status                 
бездетный      Не женат / не замужем   9.27% 
               в разводе               7.02% 
               вдовец / вдова          6.25% 
               гражданский брак        8.32% 
               женат / замужем         6.88% 
малодетный     Не женат / не замужем  11.52% 
               в разводе               7.27% 
               вдовец / вдова          9.52% 
               гражданский брак       11.11% 
               женат / замужем         8.60% 
многодетный    Не женат / не замужем  20.00% 
               в разводе               8.33% 
               вдовец / вдова          0.00% 
               гражданский брак       12.12% 
               женат / замужем         7.02%

**Вывод** 

Люди в группах 'Не женат / не замужем' и 'гражданский брак' с различным количеством детей имееют наибольший процент задолженостей по кредиту. Самый большой процент задолженостей по кредиту (20%) в группе 'Не женат / не замужем' и количество детей 3 и более. 

- Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

In [40]:
data.groupby('total_income_group')['debt'].agg(['count', 'sum', my_mean])

,count,sum,my_mean
total_income_group,,,
высокий уровень дохода,5144,361,7.02%
низкий уровень дохода,126,10,7.94%
средний доход кат.1,681,39,5.73%
средний доход кат.2,3659,305,8.34%
средний доход кат.3,5601,469,8.37%
средний доход кат.4,6314,557,8.82%


**Вывод**

Интересно, что люди с доходом от 40000-60000 имееют наименьший процент задолженостей по кредиту (5,73%). Наибольший люди с доходом 140000-200000 - 9,02%

In [48]:
data_pivot_common_status = data.pivot_table(index=['family_status','total_income_group', 'children_group'],
                                            values='debt', aggfunc='mean')
print(data_pivot_common_status['debt'].max())
display(data_pivot_common_status[data_pivot_common_status['debt'] == 0.5])
#display(data_pivot_common_status[data_pivot_common_status['debt'] == 0])
display(data_pivot_common_status)

0.5


debt
family_status         total_income_group  children_group      
Не женат / не замужем средний доход кат.2 многодетный      0.5
                      средний доход кат.4 многодетный      0.5
гражданский брак      средний доход кат.1 многодетный      0.5

debt
family_status         total_income_group     children_group          
Не женат / не замужем высокий уровень дохода бездетный       0.088652
                                             малодетный      0.097561
                                             многодетный     0.000000
                      низкий уровень дохода  бездетный       0.285714
                      средний доход кат.1    бездетный       0.033333
...                                                               ...
женат / замужем       средний доход кат.3    малодетный      0.101164
                                             многодетный     0.045455
                      средний доход кат.4    бездетный       0.075219
                                             малодетный      0.080444
                                             многодетный     0.100000

[81 rows x 1 columns]

**Вывод**

Наиболее интересными оказались люди из группы 'Не женат / не замужем' с количеством детей более 3 и доходом 60000-10000 и доходом 140000-20000, люди в гражданском браке с количеством детей больше 3 и доходом 40000-60000. Процент задолженности по кредиту среди таких людей составляет 50%. Наиболее надежными платтельщиками являются люди из категории вдовец / вдова.

- Как разные цели кредита влияют на его возврат в срок?

In [49]:
data.groupby('goal_credit')['debt'].agg(['count', 'sum', my_mean])

,count,sum,my_mean
goal_credit,,,
автомобиль,4315,403,9.34%
недвижимость,10228,747,7.30%
образование,4022,370,9.20%
ремонт,612,35,5.72%
свадьба,2348,186,7.92%


**Вывод**

Наибольший процент задолженостей по кредиту имеют люди взявшие его на образование.

## Общий вывод

* Больший процент задолженности по кредиту имеют люди с 1-2 детьми.
* Люди из группы `вдова/вдовец` имееют наименьший процент задолжености по кредиту. Люди из группы `женат/замужем` имеют практически такой же процент задолжености по кредиту как и люди из группы `в разводе`.
* Люди с низким уровнем дохода имееют наименьший процент задолженостей по кредиту (6,1%). 

Категорий самых безответственных людей:
1. Малодетные. Процент невозврата - 9,27%
2. Не женат/ не замужем. Процент невозврата - 9,75%
3. Люди с доходом 140000-200000. Процент невозврата - 9,02%
4. Люди, берущие кредит на образование. Процент невозврата - 9,22%


Надежные клиенты:
1. Бездетные. Процент невозврата - 7,54%
2. Вдова/вдовец. Процент невозврата - 6,57%
3. Люди с доходом 40000-60000. Процент невозврата - 5,73%
4. Люди, берущие кредит на ремонт. Процент невозврата - 5,77%